In [1]:
import itertools
x = ['RELIANCE.NS', 'TATASTEEL.NS', 'HDFCBANK.NS', 'INFY.NS', 'BAJAJ-AUTO.NS', 'ICICIBANK.NS', 'ITC.NS',  'UPL.NS', 'ONGC.NS', 'HINDALCO.NS', 'TITAN.NS', 'COALINDIA.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS', 'GRASIM.NS', 'JSWSTEEL.NS']
y = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOGE-USD', 'SOL1-USD', 'LTC-USD', 'BNB-USD', 'AVAX-USD', 'UNI3-USD', 'DOT1-USD', 'SUSHI-USD', 'LINK-USD', 'XRP-USD', 'ALGO-USD', 'EOS-USD', 'XTZ-USD', 'FIL-USD', 'ATOM1-USD', 'TRX-USD', 'MATIC-USD', 'CHZ-USD', 'FTT1-USD', 'NEAR-USD', 'SAND-USD', 'WAVES-USD', 'IOST-USD', 'DASH-USD', 'STORJ-USD', 'ZEC-USD', 'KSM-USD']
z = ['GC=F', 'SI=F', 'HG=F', 'PL=F', 'PA=F', 'CL=F', 'NG=F', 'RB=F', 'HO=F', 'BZ=F', 'KC=F', 'SB=F', 'CT=F', 'HE=F', 'LE=F', 'GOLD']

In [2]:
import pandas as pd
nf = pd.read_csv('nifty50.csv')
print(nf.columns)
print((nf['Symbol'] + '.NS').tolist())

Index(['Company Name', 'Industry', 'Symbol', 'Series', 'ISIN Code'], dtype='object')
['ADANIENT.NS', 'ADANIPORTS.NS', 'APOLLOHOSP.NS', 'ASIANPAINT.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BPCL.NS', 'BHARTIARTL.NS', 'BRITANNIA.NS', 'CIPLA.NS', 'COALINDIA.NS', 'DIVISLAB.NS', 'DRREDDY.NS', 'EICHERMOT.NS', 'GRASIM.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'HEROMOTOCO.NS', 'HINDALCO.NS', 'HINDUNILVR.NS', 'HDFC.NS', 'ICICIBANK.NS', 'ITC.NS', 'INDUSINDBK.NS', 'INFY.NS', 'JSWSTEEL.NS', 'KOTAKBANK.NS', 'LT.NS', 'M&M.NS', 'MARUTI.NS', 'NTPC.NS', 'NESTLEIND.NS', 'ONGC.NS', 'POWERGRID.NS', 'RELIANCE.NS', 'SBILIFE.NS', 'SBIN.NS', 'SUNPHARMA.NS', 'TCS.NS', 'TATACONSUM.NS', 'TATAMOTORS.NS', 'TATASTEEL.NS', 'TECHM.NS', 'TITAN.NS', 'UPL.NS', 'ULTRACEMCO.NS', 'WIPRO.NS']


In [3]:
print(len(x),len(y),len(z))

16 30 16


In [4]:
import yfinance as yf
import numpy as np
import pandas as pd

# # Download historical stock data for a specific ticker
# ticker = "AAPL"
# data = yf.download(ticker, start="2018-01-01", end="2023-05-10")

# Define functions to calculate technical indicators
def MACD(df, fast=12, slow=26, signal=9):
    exp1 = df["Close"].ewm(span=fast, adjust=False).mean()
    exp2 = df["Close"].ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal

def ATR(df, n=14):
    high = df["High"]
    low = df["Low"]
    close = df["Close"]
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(n).mean()
    return atr

def normalized_ATR(df, n=14):
    atr = ATR(df, n)
    natr = atr / df["Close"] * 100
    return natr

def momentum(df, n=10):
    return df["Close"].diff(n)

def CO(df):
    adl = (2*df["Close"]-df["High"]-df["Low"])/(df["High"]-df["Low"])*df["Volume"]
    ema3 = adl.ewm(span=3, adjust=False).mean()
    ema10 = adl.ewm(span=10, adjust=False).mean()
    co = ema3 - ema10
    return co

def OBV(df):
    vol = df["Volume"]
    change = np.where(df["Close"].diff() > 0, 1, -1)
    obv = (vol * change).cumsum()
    return obv

# def MFI(df, n=14):
#     typical_price = (df["High"] + df["Low"] + df["Close"]) / 3
#     money_flow = typical_price * df["Volume"]
#     pos_flow = np.where(typical_price.diff() > 0, money_flow, 0)
#     neg_flow = np.where(typical_price.diff() < 0, money_flow, 0)
#     pos_mf = pos_flow.rolling(n).sum()
#     neg_mf = neg_flow.rolling(n).sum()
#     mf_ratio = pos_mf / neg_mf
#     mfi = 100 - (100 / (1 + mf_ratio))
#     return mfi
# def MFI(df, n=14):
#     # Calculate the typical price
#     typical_price = (df["High"] + df["Low"] + df["Close"]) / 3
#     # Calculate the raw money flow
#     raw_money_flow = typical_price * df["Volume"]
#     # Calculate the money flow ratio
#     positive_flow = np.where(typical_price > typical_price.shift(), raw_money_flow, 0)
#     negative_flow = np.where(typical_price < typical_price.shift(), raw_money_flow, 0)
#     positive_mf = pd.Series(positive_flow).rolling(window=n, min_periods=0).sum()
#     negative_mf = pd.Series(negative_flow).rolling(window=n, min_periods=0).sum()
#     money_flow_ratio = positive_mf / negative_mf
#     # Calculate the money flow index
#     money_flow_index = 100 - (100 / (1 + money_flow_ratio))
#     return money_flow_index


def MFI(df):
    # Calculate the Money Flow Index
    typical_price = (df['High'] + df['Low'] + df['Close']) / 3
    raw_money_flow = typical_price * df['Volume']
    up_flow = np.where(typical_price > typical_price.shift(1), raw_money_flow, 0)
    down_flow = np.where(typical_price < typical_price.shift(1), raw_money_flow, 0)
    positive_money_flow = np.sum(up_flow)
    negative_money_flow = np.sum(down_flow)
    money_ratio = np.divide(positive_money_flow, negative_money_flow, where=negative_money_flow!=0)
    mfi = np.where(negative_money_flow==0, 100, 100 - (100 / (1 + money_ratio)))
    return mfi



# def HTDCP(df):
#     cycle, trend = pd.core.window.HammingWindow(window_len=11).frequency_response()*100
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda x: np.abs(pd.Series(x).hilbert().real))).ewm(span=21, adjust=False).mean()
#     phase = np.degrees(np.arctan((hilbert - hilbert.shift(1)) / 0.0001))
#     dcp = ((phase.rolling(14).sum() / 14) - 90) * (-1)
#     return dcp
from scipy.signal import hilbert
from scipy.signal.windows import hamming

def HTDCP(df):
    # Calculate the Hilbert Transform Dominant Cycle Phase
    analytic_signal = hilbert(df["Close"])
    instantaneous_phase = np.degrees(np.unwrap(np.angle(analytic_signal)))
    hilbert_transform = np.abs(analytic_signal)
    hilbert_envelope = pd.Series(hilbert_transform).rolling(window=31, center=True, min_periods=1).mean()
    hilbert_smoothed = pd.Series(hilbert_envelope).rolling(window=11, center=True, min_periods=1).apply(lambda x: np.convolve(x, hamming(11), mode='same'), raw=True)
    hilbert_smoothed /= np.sum(hamming(11))
    hilbert_difference = np.abs(hilbert_smoothed.diff())
    hilbert_smoothed_numpy = np.asarray(hilbert_smoothed)  # convert to numpy array
    dcphase = np.degrees(np.unwrap(np.angle(pd.Series(hilbert_smoothed_numpy * np.exp(-1j * instantaneous_phase)).rolling(window=31, center=True).mean())))
    return dcphase




# def HTS(df, n=14):
#     # Extract the analytic signal using the Hilbert Transform
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda x: pd.Series(x).hilbert()))
#     # Calculate the instantaneous amplitude and phase
#     inst_amplitude = np.abs(hilbert)
#     inst_phase = np.degrees(np.unwrap(np.angle(hilbert)))
#     # Calculate the sine wave
#     sinewave = np.sin(np.radians(inst_phase))
#     # Smooth the sine wave using an EMA
#     hts = pd.Series(sinewave).ewm(span=n, min_periods=n).mean()
#     return hts
from scipy.signal import hilbert

def HTS(df):
    # Calculate the Hilbert Transform Sinewave
    analytic_signal = hilbert(df['Close'])
    amplitude_envelope = np.abs(analytic_signal)
    instant_phase = np.unwrap(np.angle(analytic_signal))
    sinewave = np.sin(instant_phase)
    return sinewave


# def HTTMM(df):
#     # Calculate the Hilbert Transform Trend Market Mode
#     imf = pd.core.window.HammingWindow(window_len=11).frequency_response()*100
#     hilbert = pd.Series(pd.Series.rolling(df["Close"], window=31, center=True).apply(lambda x: np.abs(pd.Series(x).hilbert().real))).ewm(span=21, adjust=False).mean()
#     smoothed_imf = pd.Series(hilbert).ewm(span=14, adjust=False).mean()
#     trend = np.where(smoothed_imf > smoothed_imf.shift(), 1, -1)
#     return trend


# from scipy.signal import hilbert, chirp, find_peaks, peak_widths, welch, windows

# def HTTMM(df):
#     # Calculate the Hilbert Transform Trend Market Mode
#     instantaneous_phase = np.unwrap(np.angle(hilbert(df['Close'])))
#     inst_period = np.diff(instantaneous_phase)
#     inst_period = np.insert(inst_period, 0, inst_period[0])
#     inst_frequency = np.divide(1, inst_period)
#     frequency = welch(df['Close'], window='hamming', nperseg=len(df['Close']))[0]
#     peak_ind, _ = find_peaks(frequency, prominence=0.1)
#     widths = peak_widths(frequency, peak_ind, rel_height=0.5)
#     dominant_period = np.mean(widths[0] / len(frequency))
#     trend_market_mode = np.mod(np.divide(360, dominant_period) * instantaneous_phase, 360)
#     return trend_market_mode


from scipy.signal import hilbert, chirp, find_peaks, peak_widths, welch, windows

def HTTMM(df):
    if df.empty:
        return pd.Series()
    
    # Calculate the Hilbert Transform Trend Market Mode
    instantaneous_phase = np.unwrap(np.angle(hilbert(df['Close'])))
    inst_period = np.diff(instantaneous_phase)
    inst_period = np.insert(inst_period, 0, inst_period[0])
    inst_frequency = np.divide(1, inst_period)
    frequency = welch(df['Close'], window='hamming', nperseg=len(df['Close']))[0]
    peak_ind, _ = find_peaks(frequency, prominence=0.1)
    widths = peak_widths(frequency, peak_ind, rel_height=0.5)
    dominant_period = np.mean(widths[0] / len(frequency))
    trend_market_mode = np.mod(np.divide(360, dominant_period) * instantaneous_phase, 360)
    return trend_market_mode

# # Create a new dataframe to store the technical indicators
# features = pd.DataFrame(index=data.index)

# # Calculate the technical indicators
# features["macd"], features["signal"] = MACD(data)
# features["atr"] = ATR(data)
# features["natr"] = normalized_ATR(data)
# features["momentum"] = momentum(data)
# features["co"] = CO(data)
# features["obv"] = OBV(data)
# features["mfi"] = MFI(data)
# features["dcp"] = HTDCP(data)
# features["hts"] = HTS(data)
# features["httmm"] = HTTMM(data)

# # Print the resulting dataframe
# print(features.head())


In [5]:

import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# Define the tickers and number of y in the portfolio
# tickers = ['RELIANCE.NS', 'TCS.NS', 'HDFCBANK.NS', 'HINDUNILVR.NS', 'INFY.NS', 'ITC.NS', 'KOTAKBANK.NS', 'BAJFINANCE.NS', 'HDFC.NS', 'BSESN.BO', 'BTC-USD', 'ETH-USD', 'XRP-USD', 'LTC-USD', 'ADA-USD', 'GC=F', 'SI=F', 'CL=F', 'NG=F', 'PL=F']
# stocks = list(itertools.chain(x,y,z))
# # # ys=x
# # crypto=y
# # commod=z 
# n_y = len(y)
data_stock = {}
data_crypto={}
data_commodity={}
for stock in x:
    data_stock[stock] = yf.download(stock, period='2y',interval='1d')
# rate of return over 2 days
for stock in x:
    data_stock[stock]['return_2d'] = (data_stock[stock]['Close'] / data_stock[stock]['Open']) - 1
    #new
for stock in x:
    data_stock[stock]['macd'], data_stock[stock]['signal'] = MACD(data_stock[stock])

for stock in x:
    data_stock[stock]["atr"] = ATR(data_stock[stock])

    data_stock[stock]["natr"] = normalized_ATR(data_stock[stock])
    data_stock[stock]["momentum"] = momentum(data_stock[stock])
    data_stock[stock]["co"] = CO(data_stock[stock])
    data_stock[stock]["obv"] = OBV(data_stock[stock])
    data_stock[stock]["mfi"] = MFI(data_stock[stock])
    # data_stock[stock]["dcp"] = HTDCP(data_stock[stock])
    data_stock[stock]["hts"] = HTS(data_stock[stock])
    # data_stock[stock]["httmm"] = HTTMM(data_stock[stock])

# movig avg 10 and 5 days
for stock in x:
  data_stock[stock]['ma_10d'] = data_stock[stock]['Close'].rolling(window=10, min_periods=1).mean()
  data_stock[stock]['ma_5d'] = data_stock[stock]['Close'].rolling(window=5, min_periods=1).mean()
    
# volatility
for stock in x:
  data_stock[stock]['volatility'] = np.log(data_stock[stock]['Close'] / data_stock[stock]['Close'].shift(1)).rolling(window=10).std() * np.sqrt(252)
# volume
for stock in x:
  data_stock[stock]['volume'] = data_stock[stock]['Volume']
# rsi
for stock in x:
  delta = data_stock[stock]['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window=14, min_periods=1).mean()
  avg_loss = loss.rolling(window=14, min_periods=1).mean()
  rs = avg_gain / avg_loss
  data_stock[stock]['rsi'] = 100 - (100 / (1 + rs))
    
# psychological index
for stock in x:
  n = 14
  data_stock[stock]['up_days'] = data_stock[stock]['Close'] > data_stock[stock]['Close'].shift(1)
  data_stock[stock]['Nup'] = data_stock[stock]['up_days'].rolling(window=n).sum()
  data_stock[stock]['N'] = n
  data_stock[stock]['psychological_index'] = data_stock[stock]['Nup'] / data_stock[stock]['N']
for stock in x:
  data_stock[stock]['return_prev_day'] = data_stock[stock]['return_2d'].shift(1)
  data_stock[stock]['return_prev_week'] = data_stock[stock]['return_2d'].shift(7)
  data_stock[stock]['return_prev_fortnight'] = data_stock[stock]['return_2d'].shift(15)
#concatenate
df_stock = pd.concat(data_stock, keys=x)

2023-07-03 10:47:50.670098: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
df_stock.shape

(7888, 28)

In [7]:
for r in x:
  print(df_stock.loc[r].shape)

(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)
(493, 28)


In [8]:
for stock in y:
    data_crypto[stock] = yf.download(stock, period='2y',interval='1d')
# rate of return over 2 days
for stock in y:
    data_crypto[stock]['return_2d'] = (data_crypto[stock]['Close'] / data_crypto[stock]['Open']) - 1


    #new
for stock in y:
    data_crypto[stock]['macd'], data_crypto[stock]['signal'] = MACD(data_crypto[stock])

for stock in y:
    data_crypto[stock]["atr"] = ATR(data_crypto[stock])

    data_crypto[stock]["natr"] = normalized_ATR(data_crypto[stock])
    data_crypto[stock]["momentum"] = momentum(data_crypto[stock])
    data_crypto[stock]["co"] = CO(data_crypto[stock])
    data_crypto[stock]["obv"] = OBV(data_crypto[stock])
    data_crypto[stock]["mfi"] = MFI(data_crypto[stock])
    # data_crypto[stock]["dcp"] = HTDCP(data_crypto[stock])
    data_crypto[stock]["hts"] = HTS(data_crypto[stock])
    # data_crypto[stock]["httmm"] = HTTMM(data_crypto[stock])

# movig avg 10 and 5 days
for stock in y:
  data_crypto[stock]['ma_10d'] = data_crypto[stock]['Close'].rolling(window=10, min_periods=1).mean()
  data_crypto[stock]['ma_5d'] = data_crypto[stock]['Close'].rolling(window=5, min_periods=1).mean()
    
# volatility
for stock in y:
  data_crypto[stock]['volatility'] = np.log(data_crypto[stock]['Close'] / data_crypto[stock]['Close'].shift(1)).rolling(window=10).std() * np.sqrt(252)
# volume
for stock in y:
  data_crypto[stock]['volume'] = data_crypto[stock]['Volume']
# rsi
for stock in y:
  delta = data_crypto[stock]['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window=14, min_periods=1).mean()
  avg_loss = loss.rolling(window=14, min_periods=1).mean()
  rs = avg_gain / avg_loss
  data_crypto[stock]['rsi'] = 100 - (100 / (1 + rs))
    
# psychological index
for stock in y:
  n = 14
  data_crypto[stock]['up_days'] = data_crypto[stock]['Close'] > data_crypto[stock]['Close'].shift(1)
  data_crypto[stock]['Nup'] = data_crypto[stock]['up_days'].rolling(window=n).sum()
  data_crypto[stock]['N'] = n
  data_crypto[stock]['psychological_index'] = data_crypto[stock]['Nup'] / data_crypto[stock]['N']
for stock in y:
  data_crypto[stock]['return_prev_day'] = data_crypto[stock]['return_2d'].shift(1)
  data_crypto[stock]['return_prev_week'] = data_crypto[stock]['return_2d'].shift(7)
  data_crypto[stock]['return_prev_fortnight'] = data_crypto[stock]['return_2d'].shift(15)
#concatenate
df_crypto = pd.concat(data_crypto, keys=y)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [9]:
df_crypto

Open          High           Low         Close   
        Date                                                                 
BTC-USD 2021-06-14  39016.968750  40978.363281  38757.285156  40218.476562  \
        2021-06-15  40427.167969  41295.269531  39609.468750  40406.269531   
        2021-06-16  40168.691406  40516.777344  38176.035156  38347.062500   
        2021-06-17  38341.421875  39513.671875  37439.675781  38053.503906   
        2021-06-18  38099.476562  38187.261719  35255.855469  35787.246094   
...                          ...           ...           ...           ...   
KSM-USD 2023-06-10     25.315851     25.315851     20.161182     21.742119   
        2023-06-11     21.742035     22.271929     21.470505     21.899277   
        2023-06-12     21.898230     22.898956     21.578314     22.642050   
        2023-06-13     22.644110     23.082644     22.055384     22.398096   
        2023-06-14     22.383991     22.495855     22.261364     22.387413   

                       Adj Close       Volume  return_2d        macd   
        Date                                                           
BTC-USD 2021-06-14  40218.476562  43148914673   0.030794    0.000000  \
        2021-06-15  40406.269531  46420149185  -0.000517   14.980636   
        2021-06-16  38347.062500  39211635100  -0.045349 -137.720346   
        2021-06-17  38053.503906  37096670047  -0.007509 -279.206184   
        2021-06-18  35787.246094  36200887275  -0.060689 -567.659174   
...                          ...          ...        ...         ...   
KSM-USD 2023-06-10     21.742119     20090814  -0.141166   -0.606825   
        2023-06-11     21.899277      7273967   0.007232   -0.849594   
        2023-06-12     22.642050      6663312   0.033967   -0.970862   
        2023-06-13     22.398096      7869541  -0.010864   -1.074270   
        2023-06-14     22.387413      7455495   0.000153   -1.143898   

                        signal       atr  ...  volatility       volume   
        Date                              ...                            
BTC-USD 2021-06-14    0.000000       NaN  ...         NaN  43148914673  \
        2021-06-15    2.996127       NaN  ...         NaN  46420149185   
        2021-06-16  -25.147168       NaN  ...         NaN  39211635100   
        2021-06-17  -75.958971       NaN  ...         NaN  37096670047   
        2021-06-18 -174.299012       NaN  ...         NaN  36200887275   
...                        ...       ...  ...         ...          ...   
KSM-USD 2023-06-10   -0.397399  1.532975  ...    0.924919     20090814   
        2023-06-11   -0.487838  1.512293  ...    0.933409      7273967   
        2023-06-12   -0.584443  1.566698  ...    0.957758      6663312   
        2023-06-13   -0.682408  1.600145  ...    0.930046      7869541   
        2023-06-14   -0.774706  1.545670  ...    0.920273      7455495   

                           rsi  up_days  Nup   N  psychological_index   
        Date                                                            
BTC-USD 2021-06-14         NaN    False  NaN  14                  NaN  \
        2021-06-15  100.000000     True  NaN  14                  NaN   
        2021-06-16    8.357497    False  NaN  14                  NaN   
        2021-06-17    7.391798    False  NaN  14                  NaN   
        2021-06-18    3.906806    False  NaN  14                  NaN   
...                        ...      ...  ...  ..                  ...   
KSM-USD 2023-06-10   32.634420    False  7.0  14             0.500000   
        2023-06-11   30.035452     True  7.0  14             0.500000   
        2023-06-12   34.916812     True  8.0  14             0.571429   
        2023-06-13   34.023802    False  7.0  14             0.500000   
        2023-06-14   32.264157    False  6.0  14             0.428571   

                    return_prev_day  return_prev_week  return_prev_fortnight  
        Date                                                                  
BTC-U

In [10]:
for r in y:
  print(r,df_crypto.loc[r].shape)

BTC-USD (731, 28)
ETH-USD (731, 28)
ADA-USD (731, 28)
DOGE-USD (731, 28)
SOL1-USD (385, 28)
LTC-USD (731, 28)
BNB-USD (731, 28)
AVAX-USD (731, 28)
UNI3-USD (1, 28)
DOT1-USD (1, 28)
SUSHI-USD (731, 28)
LINK-USD (731, 28)
XRP-USD (731, 28)
ALGO-USD (731, 28)
EOS-USD (731, 28)
XTZ-USD (731, 28)
FIL-USD (731, 28)
ATOM1-USD (1, 28)
TRX-USD (731, 28)
MATIC-USD (731, 28)
CHZ-USD (731, 28)
FTT1-USD (1, 28)
NEAR-USD (731, 28)
SAND-USD (731, 28)
WAVES-USD (731, 28)
IOST-USD (731, 28)
DASH-USD (731, 28)
STORJ-USD (731, 28)
ZEC-USD (731, 28)
KSM-USD (731, 28)


In [11]:
for stock in z:
    data_commodity[stock] = yf.download(stock, period='2y',interval='1d')
# rate of return over 2 days
for stock in z:
    data_commodity[stock]['return_2d'] = (data_commodity[stock]['Close'] / data_commodity[stock]['Open']) - 1


    #new
for stock in z:
    data_commodity[stock]['macd'], data_commodity[stock]['signal'] = MACD(data_commodity[stock])

for stock in z:
    data_commodity[stock]["atr"] = ATR(data_commodity[stock])

    data_commodity[stock]["natr"] = normalized_ATR(data_commodity[stock])
    data_commodity[stock]["momentum"] = momentum(data_commodity[stock])
    data_commodity[stock]["co"] = CO(data_commodity[stock])
    data_commodity[stock]["obv"] = OBV(data_commodity[stock])
    data_commodity[stock]["mfi"] = MFI(data_commodity[stock])
    # data_commodity[stock]["dcp"] = HTDCP(data_commodity[stock])
    data_commodity[stock]["hts"] = HTS(data_commodity[stock])
    # data_commodity[stock]["httmm"] = HTTMM(data_commodity[stock])

# movig avg 10 and 5 days
for stock in z:
  data_commodity[stock]['ma_10d'] = data_commodity[stock]['Close'].rolling(window=10, min_periods=1).mean()
  data_commodity[stock]['ma_5d'] = data_commodity[stock]['Close'].rolling(window=5, min_periods=1).mean()
    
# volatility
for stock in z:
  data_commodity[stock]['volatility'] = np.log(data_commodity[stock]['Close'] / data_commodity[stock]['Close'].shift(1)).rolling(window=10).std() * np.sqrt(252)
# volume
for stock in z:
  data_commodity[stock]['volume'] = data_commodity[stock]['Volume']
# rsi
for stock in z:
  delta = data_commodity[stock]['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window=14, min_periods=1).mean()
  avg_loss = loss.rolling(window=14, min_periods=1).mean()
  rs = avg_gain / avg_loss
  data_commodity[stock]['rsi'] = 100 - (100 / (1 + rs))
    
# psychological index
for stock in z:
  n = 14
  data_commodity[stock]['up_days'] = data_commodity[stock]['Close'] > data_commodity[stock]['Close'].shift(1)
  data_commodity[stock]['Nup'] = data_commodity[stock]['up_days'].rolling(window=n).sum()
  data_commodity[stock]['N'] = n
  data_commodity[stock]['psychological_index'] = data_commodity[stock]['Nup'] / data_commodity[stock]['N']
for stock in z:
  data_commodity[stock]['return_prev_day'] = data_commodity[stock]['return_2d'].shift(1)
  data_commodity[stock]['return_prev_week'] = data_commodity[stock]['return_2d'].shift(7)
  data_commodity[stock]['return_prev_fortnight'] = data_commodity[stock]['return_2d'].shift(15)
#concatenate
df_commodity = pd.concat(data_commodity, keys=z)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [12]:
for r in z:
  print(r,df_commodity.loc[r].shape)

GC=F (507, 28)
SI=F (505, 28)
HG=F (508, 28)
PL=F (506, 28)
PA=F (505, 28)
CL=F (508, 28)
NG=F (508, 28)
RB=F (508, 28)
HO=F (508, 28)
BZ=F (508, 28)
KC=F (506, 28)
SB=F (507, 28)
CT=F (506, 28)
HE=F (500, 28)
LE=F (504, 28)
GOLD (504, 28)


In [9]:
df_filled_stock = df_stock.groupby(level=0).apply(lambda x: x.fillna(x.mean()))
df_filled_stock = df_filled_stock.fillna(0)
features = ['return_2d', 'ma_10d', 'ma_5d', 'volatility', 'rsi', 'volume','return_prev_day','return_prev_week','return_prev_fortnight','psychological_index','atr','momentum','co','obv','mfi','hts']
#features = ['return_2d']
# df_filled_crypto = df_crypto.groupby(level=0).apply(lambda x: x.fillna(x.mean()))
# df_filled_crypto = df_filled_crypto.fillna(0)
# features = ['return_2d', 'ma_10d', 'ma_5d', 'volatility', 'rsi', 'volume','return_prev_day','return_prev_week','return_prev_fortnight','psychological_index','atr','momentum','co','obv','mfi','hts']
#features = ['return_2d']
# df_filled_commodity = df_commodity.groupby(level=0).apply(lambda x: x.fillna(x.mean()))
# df_filled_commodity = df_filled_commodity.fillna(0)
# features = ['return_2d', 'ma_10d', 'ma_5d', 'volatility', 'rsi', 'volume','return_prev_day','return_prev_week','return_prev_fortnight','psychological_index','atr','momentum','co','obv','mfi','hts']
#features = ['return_2d']

In [10]:
df_filled_stock.loc["RELIANCE.NS"].columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'return_2d',
       'macd', 'signal', 'atr', 'natr', 'momentum', 'co', 'obv', 'mfi', 'hts',
       'ma_10d', 'ma_5d', 'volatility', 'volume', 'rsi', 'up_days', 'Nup', 'N',
       'psychological_index', 'return_prev_day', 'return_prev_week',
       'return_prev_fortnight'],
      dtype='object')

In [11]:
data_stock = df_filled_stock[features] #normal rll features

In [12]:
data_stock.columns

Index(['return_2d', 'ma_10d', 'ma_5d', 'volatility', 'rsi', 'volume',
       'return_prev_day', 'return_prev_week', 'return_prev_fortnight',
       'psychological_index', 'atr', 'momentum', 'co', 'obv', 'mfi', 'hts'],
      dtype='object')

In [13]:
print(data_stock.shape)

(7888, 16)


In [14]:
data_stock.loc['UPL.NS'].shape
# len(x)

(493, 16)

In [15]:


# train_data_stock=pd.DataFrame(columns=features)
# test_data_stock=pd.DataFrame(columns=features)
# for i in x:
#   train_data_stock=train_data_stock.append(data_stock.loc[i][:400])
#   test_data_stock=test_data_stock.append(data_stock.loc[i][400:])

x_excluded_train=[]
x_excluded_test=[]
#Define the input sequences and corresponding labels:
def create_sequences_train(data, seq_length,names):
    X = []
    y = []
    for j in names:
      if(len(data.loc[j][:400])<seq_length):
        x_excluded_train.append(j)
        continue
      for i in range(seq_length, len(data.loc[j][:400])):
          X.append(data.loc[j][i-seq_length:i])
          y.append(data.loc[j]['return_2d'][i])
    return np.array(X), np.array(y)
def create_sequences_test(data, seq_length,names):
    X = []
    y = []
    for j in names:
      x_temp=[]
      y_temp=[]
      if(len(data.loc[j][400:])<seq_length):
        x_excluded_test.append(j)
        continue
      for i in range(seq_length,len(data.loc[j][400:])):
          x_temp.append(data.loc[j][len(data.loc[j][:400])+i-seq_length:len(data.loc[j][:400])+i])
          y_temp.append(data.loc[j]['return_2d'][len(data.loc[j][:400])+i])
      X.append(np.array(x_temp))
      y.append(np.array(y_temp))
    return np.array(X), np.array(y)


seq_length = 10
X_train_stock, y_train_stock = create_sequences_train(data_stock, seq_length,x)
X_test_stock, y_test_stock = create_sequences_test(data_stock, seq_length,x)

print(X_train_stock.shape)
print(X_test_stock.shape)


# Reshape the data to match the input shape expected by the LSTM model:
X_train_stock = np.reshape(X_train_stock, (X_train_stock.shape[0], X_train_stock.shape[1], len(features)))
# X_test_stock = np.reshape(X_test_stock, (X_test_stock.shape[0], X_test_stock.shape[1], len(features)))


# # for pca
# X_train_stock = np.reshape(X_train_stock, (X_train_stock.shape[0], X_train_stock.shape[1], len(features_pca)))
# X_test_stock = np.reshape(X_test_stock, (X_test_stock.shape[0], X_test_stock.shape[1], len(features_pca)))




(6240, 10, 16)
(16, 83, 10, 16)


In [22]:
X_test_stock[0].shape

(83, 10, 16)

In [26]:
X_train_stock.shape

(6240, 10, 16)

In [27]:
y_train_stock.shape

(6240,)

In [23]:
X_test_stock.shape

(16, 83, 10, 16)

In [24]:
y_test_stock.shape

(16, 83)

In [42]:
for r in z:
  print(r,data_commodity.loc[r].shape)

GC=F (507, 16)
SI=F (505, 16)
HG=F (508, 16)
PL=F (506, 16)
PA=F (505, 16)
CL=F (508, 16)
NG=F (508, 16)
RB=F (508, 16)
HO=F (508, 16)
BZ=F (508, 16)
KC=F (506, 16)
SB=F (507, 16)
CT=F (506, 16)
HE=F (500, 16)
LE=F (504, 16)
GOLD (504, 16)


In [43]:



# train_data_commodity=pd.DataFrame(columns=features)
# test_data_commodity=pd.DataFrame(columns=features)
# for i in z:
#   train_data_commodity=train_data_commodity.append(data_commodity.loc[i][:400])
#   test_data_commodity=test_data_commodity.append(data_commodity.loc[i][400:])

# #Normalize the data:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train_data_commodity = scaler.fit_transform(train_data_commodity)
# test_data_commodity = scaler.transform(test_data_commodity)

#Define the input sequences and corresponding labels:
z_excluded_train=[]
z_excluded_test=[]
def create_sequences_train(data, seq_length,names):
    X = []
    y = []
    for j in names:
      if(len(data.loc[j][:400])<seq_length):
        z_excluded_train.append(j)
        continue
      for i in range(seq_length, len(data.loc[j][:400])):
          X.append(data.loc[j][i-seq_length:i])
          y.append(data.loc[j]['return_2d'][i])
    return np.array(X), np.array(y)
def create_sequences_test(data, seq_length,names):
    X = []
    y = []
    for j in names:
      x_temp=[]
      y_temp=[]
      if(len(data.loc[j][400:])<seq_length):
        x_excluded_test.append(j)
        continue
      for i in range(seq_length,len(data.loc[j][400:])):
          x_temp.append(data.loc[j][len(data.loc[j][:400])+i-seq_length:len(data.loc[j][:400])+i])
          y_temp.append(data.loc[j]['return_2d'][len(data.loc[j][:400])+i])
      X.append(np.array(x_temp))
      y.append(np.array(y_temp))
    return np.array(X), np.array(y)

seq_length = 10
X_train_commodity, y_train_commodity = create_sequences_train(data_commodity, seq_length,z)
X_test_commodity, y_test_commodity = create_sequences_test(data_commodity, seq_length,z)
# print(X_train_commodity)

#Reshape the data to match the input shape expected by the LSTM model:
X_train_commodity = np.reshape(X_train_commodity, (X_train_commodity.shape[0], X_train_commodity.shape[1], len(features)))
# X_test_commodity = np.reshape(X_test_commodity, (X_test_commodity.shape[0], X_test_commodity.shape[1], len(features)))

# # for pca
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(features_pca)))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(features_pca)))




/var/folders/fk/093vj0zx0pvf5p2n4l0tq9hh0000gn/T/ipykernel_29034/2591770985.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X), np.array(y)


In [44]:
print(X_test_commodity[0].shape)

(97, 10, 16)


In [45]:
X_train_commodity.shape

(6240, 10, 16)

In [46]:
y_train_commodity.shape

(6240,)

In [47]:
X_test_commodity.shape

(16,)

In [48]:
y_test_commodity.shape

(16,)

In [49]:
print(z_excluded_train)
print(z_excluded_test)

[]
[]


In [50]:
data_crypto = df_filled_crypto[features] #normal rll features



In [51]:
for r in y:
  print(r,data_crypto.loc[r].shape)

BTC-USD (731, 16)
ETH-USD (731, 16)
ADA-USD (731, 16)
DOGE-USD (731, 16)
SOL1-USD (385, 16)
LTC-USD (731, 16)
BNB-USD (731, 16)
AVAX-USD (731, 16)
UNI3-USD (1, 16)
DOT1-USD (1, 16)
SUSHI-USD (731, 16)
LINK-USD (731, 16)
XRP-USD (731, 16)
ALGO-USD (731, 16)
EOS-USD (731, 16)
XTZ-USD (731, 16)
FIL-USD (731, 16)
ATOM1-USD (1, 16)
TRX-USD (731, 16)
MATIC-USD (731, 16)
CHZ-USD (731, 16)
FTT1-USD (1, 16)
NEAR-USD (731, 16)
SAND-USD (731, 16)
WAVES-USD (731, 16)
IOST-USD (731, 16)
DASH-USD (731, 16)
STORJ-USD (731, 16)
ZEC-USD (731, 16)
KSM-USD (731, 16)


In [52]:
# remove from y the 1 or 385 ones
y_exclude=['SOL1-USD','UNI3-USD','DOT1-USD','ATOM1-USD','FTT1-USD']

In [53]:
crypto_names=y

In [54]:

# train_data_crypto=pd.DataFrame(columns=features)
# test_data_crypto=pd.DataFrame(columns=features)
# for i in y:
#   train_data_crypto=train_data_crypto.append(data_crypto.loc[i][:400])
#   test_data_crypto=test_data_crypto.append(data_crypto.loc[i][400:])

# #Normalize the data:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# train_data = scaler.fit_transform(train_data)
# test_data = scaler.transform(test_data)

#Define the input sequences and corresponding labels:
def create_sequences_train(data, seq_length,names,names_exclude):
    X = []
    y = []
    for j in names :
      if(j in names_exclude or len(data.loc[j][:400])<seq_length):
        continue
      for i in range(seq_length, len(data.loc[j][:400])):
          X.append(data.loc[j][i-seq_length:i])
          y.append(data.loc[j]['return_2d'][i])
    return np.array(X), np.array(y)

def create_sequences_test(data, seq_length,names,names_exclude):

    X = []
    y = []
    for j in names:
      x_temp=[]
      y_temp=[]
      if(len(data.loc[j][400:])<seq_length):
        x_excluded_test.append(j)
        continue
      for i in range(seq_length,len(data.loc[j][400:])):
          x_temp.append(data.loc[j][len(data.loc[j][:400])+i-seq_length:len(data.loc[j][:400])+i])
          y_temp.append(data.loc[j]['return_2d'][len(data.loc[j][:400])+i])
      X.append(np.array(x_temp))
      y.append(np.array(y_temp))
    return np.array(X), np.array(y)



seq_length = 10
X_train_crypto, y_train_crypto = create_sequences_train(data_crypto, seq_length,crypto_names,y_exclude)
X_test_crypto, y_test_crypto = create_sequences_test(data_crypto, seq_length,crypto_names,y_exclude)


#Reshape the data to match the input shape expected by the LSTM model:
X_train_crypto = np.reshape(X_train_crypto, (X_train_crypto.shape[0], X_train_crypto.shape[1], len(features)))
# X_test_crypto = np.reshape(X_test_crypto, (X_test_crypto.shape[0], X_test_crypto.shape[1], len(features)))

# for pca
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], len(features_pca)))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], len(features_pca)))




In [68]:
print(len(features), len(x))
X_train_stock[0,0,:]


16 16


array([ 1.34988272e-02,  2.24489990e+03,  2.24489990e+03,  2.30075079e-01,
        5.13322774e+01,  9.38341600e+06, -5.97510246e-04, -6.13287091e-04,
       -6.24740784e-04,  5.09720177e-01,  5.25155660e+01,  5.81895472e+00,
        0.00000000e+00, -9.38341600e+06,  4.94137805e+01,  2.13117389e-01])

In [33]:
from sklearn.preprocessing import RobustScaler,StandardScaler
sc_x_stock = StandardScaler()
sc_y_stock = StandardScaler()
X_reshaped = np.reshape(X_train_stock, (X_train_stock.shape[0] * X_train_stock.shape[1], X_train_stock.shape[2]))
X_scaled = sc_x_stock.fit_transform(X_reshaped)
X_train_stock = np.reshape(X_scaled, (X_train_stock.shape[0], X_train_stock.shape[1], X_train_stock.shape[2]))

y_reshaped = np.reshape(y_train_stock, (-1, 1))
y_scaled = sc_y_stock.fit_transform(y_reshaped)
y_train_stock = np.reshape(y_scaled, (y_train_stock.shape[0], 1))

In [68]:
X_reshaped.shape, y_reshaped.shape, features

((830, 16),
 (6240, 1),
 ['return_2d',
  'ma_10d',
  'ma_5d',
  'volatility',
  'rsi',
  'volume',
  'return_prev_day',
  'return_prev_week',
  'return_prev_fortnight',
  'psychological_index',
  'atr',
  'momentum',
  'co',
  'obv',
  'mfi',
  'hts'])

In [29]:
#Define the LSTM model
model_stock = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True,input_shape=(X_train_stock.shape[1], len(features)))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
# Compile the model
model_stock.compile(optimizer='adam', loss='mse')

In [31]:
# Train the model
start=time.time()

history_stock=model_stock.fit(X_train_stock,y_train_stock, epochs=1)

end=time.time()
print("time taken: ", end-start)

model_stock.summary()


  5/195 [..............................] - ETA: 6s - loss: 1.5484

195/195 [==============================] - 11s 56ms/step - loss: 0.9455
time taken:  10.99373173713684
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (32, 10, 256)            148480    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (32, 128)                164352    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (32, 128)                 0         
                                                                 
 dense_1 (Dense)             (32, 1)                   129       
                                                                 
Total params: 312,961
Trainable params: 312,961
Non-trainable params: 0
___________

**pred**

In [44]:
x = ['RELIANCE.NS', 'TATASTEEL.NS', 'HDFCBANK.NS', 'INFY.NS', 'BAJAJ-AUTO.NS', 'ICICIBANK.NS', 'ITC.NS',  'UPL.NS', 'ONGC.NS', 'HINDALCO.NS', 'TITAN.NS', 'COALINDIA.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS', 'GRASIM.NS', 'JSWSTEEL.NS']
y = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOGE-USD', 'SOL1-USD', 'LTC-USD', 'BNB-USD', 'AVAX-USD', 'UNI3-USD', 'DOT1-USD', 'SUSHI-USD', 'LINK-USD', 'XRP-USD', 'ALGO-USD', 'EOS-USD', 'XTZ-USD', 'FIL-USD', 'ATOM1-USD', 'TRX-USD', 'MATIC-USD', 'CHZ-USD', 'FTT1-USD', 'NEAR-USD', 'SAND-USD', 'WAVES-USD', 'IOST-USD', 'DASH-USD', 'STORJ-USD', 'ZEC-USD', 'KSM-USD']
#y exclude exclude -   ****IMPPPPP*****
z = ['GC=F', 'SI=F', 'HG=F', 'PL=F', 'PA=F', 'CL=F', 'NG=F', 'RB=F', 'HO=F', 'BZ=F', 'KC=F', 'SB=F', 'CT=F', 'HE=F', 'LE=F', 'GOLD']

In [76]:
# X_reshaped = np.reshape(X_test_stock, (X_test_stock.shape[0] * X_test_stock.shape[1], X_test_stock.shape[2]))
# X_scaled = sc_x_stock.fit_transform(X_reshaped)
# X_test_stock = np.reshape(X_scaled, (X_test_stock.shape[0], X_test_stock.shape[1], X_test_stock.shape[2]))

y_reshaped = np.reshape(y_test_stock, (-1, 1))
y_scaled = sc_y_stock.transform(y_reshaped)
y_test_stock = np.reshape(y_scaled, (y_test_stock.shape[0], y_test_stock.shape[1], 1))

In [ ]:
# X_reshaped = np.reshape(X_test_crypto, (X_test_crypto.shape[0] * X_test_crypto.shape[1], X_test_crypto.shape[2]))
# X_scaled = sc_x_crypto.fit_transform(X_reshaped)
# X_test_crypto = np.reshape(X_scaled, (X_test_crypto.shape[0], X_test_crypto.shape[1], X_test_crypto.shape[2]))

# y_reshaped = np.reshape(y_test_crypto, (-1, 1))
# y_scaled = sc_y_crypto.fit_transform(y_reshaped)
# y_test_crypto = np.reshape(y_scaled, (y_test_crypto.shape[0], 1))

In [ ]:
# X_reshaped = np.reshape(X_test_commodity, (X_test_commodity.shape[0] * X_test_commodity.shape[1], X_test_commodity.shape[2]))
# X_scaled = sc_x_commodity.fit_transform(X_reshaped)
# X_test_commodity = np.reshape(X_scaled, (X_test_commodity.shape[0], X_test_commodity.shape[1], X_test_commodity.shape[2]))

# y_reshaped = np.reshape(y_test_commodity, (-1, 1))
# y_scaled = sc_y_commodity.fit_transform(y_reshaped)
# y_test_commodity = np.reshape(y_scaled, (y_test_commodity.shape[0], 1))

In [77]:
p_stock=[]
for i in range(len(X_test_stock)):

  X_reshaped = np.reshape(X_test_stock[i], (X_test_stock[i].shape[0] * X_test_stock[i].shape[1], X_test_stock[i].shape[2]))
  X_scaled = sc_x_stock.transform(X_reshaped)
  temp = np.reshape(X_scaled, (X_test_stock[i].shape[0], X_test_stock[i].shape[1], X_test_stock[i].shape[2]))

  prediction=model_stock.predict(temp)
  prediction=np.array(prediction)
  p = np.reshape(prediction, (-1, 1))
  p_inverse=sc_y_stock.inverse_transform(p)
  p_final_stock = np.reshape(p_inverse, (prediction.shape[0], 1))
  p_stock.append(p_final_stock)
p_stock=np.array(p_stock)
print(p_stock.shape) 




3/3 [==============================] - 0s 50ms/step
(16, 83, 1)


In [79]:
error

array([[[-0.49079498],
        [-0.42165684],
        [-0.27063937],
        ...,
        [ 0.08589088],
        [ 0.05591796],
        [ 0.10247793]],

       [[ 0.18031588],
        [ 0.17106266],
        [ 0.17761946],
        ...,
        [ 0.20671472],
        [ 0.20366874],
        [ 0.21309108]],

       [[ 0.02926641],
        [ 0.05024692],
        [ 0.01732888],
        ...,
        [ 0.21015637],
        [ 0.22577044],
        [ 0.23421925]],

       ...,

       [[ 0.20867556],
        [ 0.20435536],
        [ 0.18328259],
        ...,
        [ 0.19441957],
        [ 0.23272697],
        [ 0.21473106]],

       [[ 0.19834764],
        [ 0.18641334],
        [ 0.2051089 ],
        ...,
        [ 0.2002413 ],
        [ 0.20876648],
        [ 0.22878684]],

       [[ 0.16943998],
        [ 0.18051067],
        [ 0.16266849],
        ...,
        [ 0.20329342],
        [ 0.22142699],
        [ 0.21446851]]])

In [62]:
np.mean(np.abs(error))

0.20331237809978023

In [81]:
percentage_error = np.divide(error, y_test_stock, out=np.zeros_like(error), where=y_test_stock.reshape(16,83,1)!=0)
percentage_error = np.abs(percentage_error)
percentage_error = np.mean(percentage_error, axis=1)    
percentage_error

array([[1.92300416e+02],
       [2.54884321e+14],
       [6.17112539e+01],
       [3.09649937e+14],
       [1.36917054e+14],
       [1.52097175e+02],
       [1.07159399e+02],
       [1.21821315e+14],
       [6.79090332e+01],
       [6.80210852e+01],
       [1.35479766e+02],
       [1.27442179e+14],
       [6.91485967e+01],
       [6.80313064e+01],
       [1.30881043e+02],
       [4.33646563e+01]])

In [36]:
print(y_test_stock.shape)
print(p_stock.shape)

(16, 83)
(16, 83, 1)


NameError: name 'y_test_crypto' is not defined

In [51]:
opening_prices = {}
for stock_name in df_stock.index.unique():
    stock_open = df_stock.loc[stock_name, 'Open']
    opening_prices[stock_name] = stock_open
i=0
j=0

open_prices_stock=np.zeros((498,len(x)))
for stock_name, open_prices in opening_prices.items():
    open_prices_stock[i][j]=open_prices
    i=(i+1)%498
    if i==0:
      j+=1
    #print(f"Stock: {stock_name}")
    #print(open_prices)
    #print("\n")


In [52]:
open_prices_stock.shape

(498, 16)

In [53]:
open_prices_stock_t=open_prices_stock[400:]

In [54]:
import math

**returns**

3 model(lstm) - 2809357

garch + 3 model(lstm) - 2254678

y_test_stock and p_stock are the names of the arrays

In [55]:
capital=100000
portfolio_value=capital
for i in range(80):
    predicted_returns=[]
    for j in range(len(y_test_stock)):
      predicted_returns.append(p_stock[j][i])
    # for j in range(len(y_test_crypto)):
    #   predicted_returns.append(p_crypto[j][i])
    for j in range(len(y_test_commodity)):
      predicted_returns.append(p_commodity[j][i])

    predicted_returns=np.array(predicted_returns)
    # print("predicted_returns",predicted_returns)
    returns=[]
    for j in range(len(y_test_stock)):
      returns.append(y_test_stock[j][i])

    # for j in range(len(y_test_crypto)):
    #   returns.append(y_test_crypto[j][i])
    for j in range(len(y_test_commodity)):
      returns.append(y_test_commodity[j][i])
    # print("returns actual",returns)
    predicted_returns=np.array(predicted_returns)
    returns=np.array(returns)
    # print(predicted_returns.shape)
    # print(returns.shape)
    predicted_returns=np.squeeze(predicted_returns,axis=1)
    indices = np.argsort(np.abs(predicted_returns))[::-1]
    # print("indices= ",indices)
    # indices=indices[0]
    
   
    # Sort both 'a' and 'b' arrays in the same order
    returns = returns[indices]
    predicted_returns = predicted_returns[indices]
    # print(returns)

    # returns=np.nan_to_num(returns)
    # print("returns actual",returns)

    p=[]
    r=[]
    count=0
    for j in range(len(predicted_returns)):
      if(count==10):
        break
      if(np.isnan(returns[j])):
        continue
      else:
        p.append(predicted_returns[j])
        r.append(returns[j])
        count+=1

    p=np.array(p)
    r=np.array(r)
    # print(p.shape)
    print(p)
    print(r)
    # print(r.shape)
    # print("pred retuns",p)
    # print("actual retuns",r)
    weights = (p) / np.sum(np.abs(p))
    # print("weights new",weights)

    capital_each=portfolio_value*weights 
    cap_diff=0
    not_stock=[-1]
    for idx in range(10):
      if indices[idx]<16:
        capital_new=math.floor(capital_each[idx]/open_prices_stock_t[i][indices[idx]])*open_prices_stock_t[i][indices[idx]]
        cap_diff+=capital_each[idx]-capital_new
        capital_each[idx]=capital_new
      else:
        not_stock.append(idx)
    val=0
    if len(not_stock)==1:
      val+=cap_diff
    else:
      capital_each[not_stock[1]]+=cap_diff

    print(capital_each)
    print(capital_each.shape)
    print(sum(abs(capital_each)))
    # print("capital_each",capital_each)
    gain_loss=np.dot(capital_each,r)
    print("Day {0} return is {1}".format(i+1,gain_loss/portfolio_value*100))
    # print("gain loss",gain_loss)
    portfolio_value += (gain_loss+val)

    # capital = portfolio_value
    print("gain/loss:",gain_loss,end=", ")
    print("portfolio_value:",portfolio_value)


# Print the final portfolio value
print("Final portfolio value:", portfolio_value)



[ 0.06526967  0.05931824 -0.04920227 -0.0437849  -0.04018477 -0.03792926
  0.03088278 -0.03031974 -0.02999551  0.02764573]
[ 0.02253247  0.00064729 -0.00992644 -0.02615743 -0.04011007 -0.01429033
 -0.00412024 -0.03148575 -0.02938197  0.        ]
[ 14152.20019531  17702.75634742 -12702.40039062 -11251.5
  -9712.          -9149.88070726   7450.0195682   -7384.99984741
  -7425.           6669.12794113]
(10,)
103599.88499736786
Day 1 return is 1.6910341823111483
gain/loss: 1691.0341823111485, portfolio_value: 101691.03418231115
[-0.0349545   0.03391996  0.03345619 -0.03252277 -0.03057267  0.03028848
  0.02735673  0.02563044 -0.02546969 -0.0238195 ]
[-0.01388308  0.03606766  0.03749346  0.00223716 -0.03213791  0.00226615
  0.01745149  0.029375   -0.01887383 -0.02508886]
[-12191.25   16805.574  11417.108 -11098.573 -10710.     10336.109
   8368.5     8647.     -9704.    -10740.   ]
(10,)
110018.115234375
Day 2 return is 2.3590102684068124
gain/loss: 2398.9019384098015, portfolio_value: 10408

In [56]:
capital=100000
portfolio_value=capital
for i in range(80):
    predicted_returns=[]
    for j in range(len(y_test_stock)):
      predicted_returns.append(p_stock[j][i])
    # for j in range(len(y_test_crypto)):
    #   predicted_returns.append(p_crypto[j][i])
    for j in range(len(y_test_commodity)):
      predicted_returns.append(p_commodity[j][i])

    predicted_returns=np.array(predicted_returns)
    # print("predicted_returns",predicted_returns)
    returns=[]
    for j in range(len(y_test_stock)):
      returns.append(y_test_stock[j][i])

    # for j in range(len(y_test_crypto)):
    #   returns.append(y_test_crypto[j][i])
    for j in range(len(y_test_commodity)):
      returns.append(y_test_commodity[j][i])
    # print("returns actual",returns)
    predicted_returns=np.array(predicted_returns)
    returns=np.array(returns)
    # print(predicted_returns.shape)
    # print(returns.shape)
    predicted_returns=np.squeeze(predicted_returns,axis=1)
    indices = np.argsort(np.abs(predicted_returns))[::-1]
    # print("indices= ",indices)
    # indices=indices[0]
    
   
    # Sort both 'a' and 'b' arrays in the same order
    returns = returns[indices]
    predicted_returns = predicted_returns[indices]
    # print(returns)

    # returns=np.nan_to_num(returns)
    # print("returns actual",returns)

    p=[]
    r=[]
    count=0
    for j in range(len(predicted_returns)):
      if(count==10):
        break
      if(np.isnan(returns[j])):
        continue
      else:
        p.append(predicted_returns[j])
        r.append(returns[j])
        count+=1

    p=np.array(p)
    r=np.array(r)
    # print(p.shape)
    print(p)
    print(r)
    # print(r.shape)
    # print("pred retuns",p)
    # print("actual retuns",r)
    weights = (p) / np.sum(np.abs(p))
    # print("weights new",weights)

    capital_each=portfolio_value*weights 
    cap_diff=0
    not_stock=[-1]
    for idx in range(10):
      if indices[idx]<16:
        capital_new=math.floor(capital_each[idx]/open_prices_stock_t[i][indices[idx]])*open_prices_stock_t[i][indices[idx]]
        cap_diff+=capital_each[idx]-capital_new
        capital_each[idx]=capital_new
      else:
        not_stock.append(idx)
    val=0
    if len(not_stock)==1:
      val+=cap_diff
    else:
      capital_each[not_stock[1]]+=cap_diff

    print(capital_each)
    print(capital_each.shape)
    print(sum(abs(capital_each)))
    # print("capital_each",capital_each)
    gain_loss=np.dot(capital_each,r)
    print("Day {0} return is {1}".format(i+1,gain_loss/portfolio_value*100))
    # print("gain loss",gain_loss)
    portfolio_value += (gain_loss+val -400)

    # capital = portfolio_value
    print("gain/loss:",gain_loss,end=", ")
    print("portfolio_value:",portfolio_value)


# Print the final portfolio value
print("Final portfolio value:", portfolio_value)



[ 0.06526967  0.05931824 -0.04920227 -0.0437849  -0.04018477 -0.03792926
  0.03088278 -0.03031974 -0.02999551  0.02764573]
[ 0.02253247  0.00064729 -0.00992644 -0.02615743 -0.04011007 -0.01429033
 -0.00412024 -0.03148575 -0.02938197  0.        ]
[ 14152.20019531  17702.75634742 -12702.40039062 -11251.5
  -9712.          -9149.88070726   7450.0195682   -7384.99984741
  -7425.           6669.12794113]
(10,)
103599.88499736786
Day 1 return is 1.6910341823111483
gain/loss: 1691.0341823111485, portfolio_value: 101291.03418231115
[-0.0349545   0.03391996  0.03345619 -0.03252277 -0.03057267  0.03028848
  0.02735673  0.02563044 -0.02546969 -0.0238195 ]
[-0.01388308  0.03606766  0.03749346  0.00223716 -0.03213791  0.00226615
  0.01745149  0.029375   -0.01887383 -0.02508886]
[-12191.25   16843.037  11372.199 -11054.918 -10710.     10295.452
   8368.5     8647.     -9704.    -10740.   ]
(10,)
109926.3564453125
Day 2 return is 2.368003135303464
gain/loss: 2398.574865218431, portfolio_value: 103289

In [ ]:
2# import itertools
# x = ['RELIANCE.NS', 'TATASTEEL.NS', 'HDFCBANK.NS', 'INFY.NS', 'BAJAJ-AUTO.NS', 'ICICIBANK.NS', 'ITC.NS',  'UPL.NS', 'ONGC.NS', 'HINDALCO.NS', 'TITAN.NS', 'COALINDIA.NS', 'INDUSINDBK.NS', 'BAJAJFINSV.NS', 'GRASIM.NS', 'JSWSTEEL.NS']
# y = ['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOGE-USD', 'SOL1-USD', 'LTC-USD', 'BNB-USD', 'AVAX-USD', 'UNI3-USD', 'DOT1-USD', 'SUSHI-USD', 'LINK-USD', 'XRP-USD', 'ALGO-USD', 'EOS-USD', 'XTZ-USD', 'FIL-USD', 'ATOM1-USD', 'TRX-USD', 'MATIC-USD', 'CHZ-USD', 'FTT1-USD', 'NEAR-USD', 'SAND-USD', 'WAVES-USD', 'IOST-USD', 'DASH-USD', 'STORJ-USD', 'ZEC-USD', 'KSM-USD']
# z = ['GC=F', 'SI=F', 'HG=F', 'PL=F', 'PA=F', 'CL=F', 'NG=F', 'RB=F', 'HO=F', 'BZ=F', 'KC=F', 'SB=F', 'CT=F', 'HE=F', 'LE=F', 'GOLD']

GARCH

In [ ]:
# import numpy as np
# import pandas as pd
# import scipy.optimize as sco
# import statsmodels.api as sm
# garch_models = []
# garch_results = []
# forecasted_volatilities = []
# forecast_horizon=10
# for i in range(y_train_commodity.shape):
#     garch_model = sm.tsa.GARCH(1, 1)
#     garch_results.append(garch_model.fit(y_train_commodity.iloc[:, i]))
#     garch_models.append(garch_model)
#     forecasted_volatilities.append(garch_results[i].forecast(horizon=forecast_horizon).variance.values[-1])

TypeError: ignored

In [ ]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.4/916.4 kB 44.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
from scipy.optimize import minimize

# Set the predicted and actual returns for each stock
predicted_returns = np.array([0.02, 0.03, 0.01, -0.01, 0.02])
actual_returns = np.array([0.01, 0.02, 0.03, -0.02, -0.01])

# Function to calculate portfolio variance
def portfolio_variance(weights, returns):
    weighted_returns = returns * weights
    portfolio_var = np.dot(weighted_returns.T, weighted_returns)
    return portfolio_var

# GARCH model estimation
def garch_estimation(returns):
    returns=returns+1e-08
    model = arch_model(returns, vol='Garch', p=1, q=1)
    model_fit = model.fit(disp='off')
    return model_fit

# Optimize portfolio weights
def optimize_portfolio(predicted_returns, actual_returns):
    num_assets = len(predicted_returns)
    
    # Estimate GARCH parameters for each asset
    garch_models = []
    for i in range(num_assets):
        model_fit = garch_estimation(actual_returns[i])
        garch_models.append(model_fit)
    
    # Set optimization constraints
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
    bounds = tuple((0, 1) for _ in range(num_assets))
    
    # Initial weights (equal weight allocation)
    init_weights = np.ones(num_assets) / num_assets
    
    # Define objective function (minimize portfolio variance)
    objective = lambda weights: portfolio_variance(weights, predicted_returns)
    
    # Optimize portfolio weights using scipy minimize function
    optimized_weights = minimize(objective, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return optimized_weights.x

# Run the portfolio optimization
optimized_weights = optimize_portfolio(predicted_returns, actual_returns)

# Display the optimized weights for each asset
for i in range(len(optimized_weights)):
    print(f"Asset {i+1}: {optimized_weights[i]:.2f}")


Asset 1: 0.20
Asset 2: 0.20
Asset 3: 0.20
Asset 4: 0.20
Asset 5: 0.20


/usr/local/lib/python3.10/dist-packages/arch/univariate/volatility.py:1196: RuntimeWarning: invalid value encountered in double_scalars
  scale = np.mean(resids**2) / (target ** (2.0 / power))
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:451: RuntimeWarning: divide by zero encountered in log
  lls = -0.5 * (log(2 * pi) + log(sigma2) + resids ** 2.0 / sigma2)
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:451: RuntimeWarning: invalid value encountered in true_divide
  lls = -0.5 * (log(2 * pi) + log(sigma2) + resids ** 2.0 / sigma2)
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:451: RuntimeWarning: divide by zero encountered in true_divide
  lls = -0.5 * (log(2 * pi) + log(sigma2) + resids ** 2.0 / sigma2)
/usr/local/lib/python3.10/dist-packages/arch/univariate/distribution.py:451: RuntimeWarning: invalid value encountered in add
  lls = -0.5 * (log(2 * pi) + log(sigma2) + resids ** 2.0 / sigma2)
/usr/local/li

In [ ]:
print(predicted_returns.shape)
print(actual_returns.shape)

(5,)
(5,)


In [ ]:
capital=100000
portfolio_value=capital
import warnings
warnings.filterwarnings("ignore")
for i in range(80):
    predicted_returns=[]
    for j in range(len(y_test_stock)):
      predicted_returns.append(p_stock[j][i])
    for j in range(len(y_test_crypto)):
      predicted_returns.append(p_crypto[j][i])
    for j in range(len(y_test_commodity)):
      predicted_returns.append(p_commodity[j][i])

    predicted_returns=np.array(predicted_returns)
    # print("predicted_returns",predicted_returns)
    returns=[]
    for j in range(len(y_test_stock)):
      returns.append(y_test_stock[j][i])

    for j in range(len(y_test_crypto)):
      returns.append(y_test_crypto[j][i])
    for j in range(len(y_test_commodity)):
      returns.append(y_test_commodity[j][i])
    # print("returns actual",returns)
    predicted_returns=np.array(predicted_returns)
    returns=np.array(returns)
    # print(predicted_returns.shape)
    # print(returns.shape)
    predicted_returns=np.squeeze(predicted_returns,axis=1)
    # print(predicted_returns.shape)
    # print(returns.shape)
    optimized_weights = optimize_portfolio(predicted_returns,returns)
    optimized_weights=np.array(optimized_weights)
    capital_each=portfolio_value*weights
    print(capital_each)
    print(capital_each.shape)
    print(sum(abs(capital_each)))
    # print("capital_each",capital_each)
    gain_loss=np.dot(capital_each,r)
    print("Day {0} return is {1}".format(i+1,gain_loss/portfolio_value*100))
    # print("gain loss",gain_loss)
    portfolio_value += gain_loss

    # capital = portfolio_value
    print("gain/loss:",gain_loss,end=", ")
    print("portfolio_value:",portfolio_value)


# Print the final portfolio value
print("Final portfolio value:", portfolio_value)



/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

[-13963.18376064  13278.46497297 -12689.52935934  11040.48192501
 -10942.36597419  10191.13585353   7414.88710046  -7167.10463166
   6717.04933047  -6595.79187632]
(10,)
99999.99478459358
Day 1 return is 1.652480291469448
gain/loss: 1652.4802914694483, portfolio_value: 101652.48029146944


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

[-14193.922   13497.889  -12899.221   11222.923  -11123.186   10359.542
   7537.4165  -7285.5396   6828.047   -6704.7856]
(10,)
101652.47021484375
Day 2 return is 1.6524802138804295
gain/loss: 1679.7871237352358, portfolio_value: 103332.26741520468


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

...

/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

[-50140.293  47681.54  -45566.734  39645.184 -39292.86   36595.273
  26626.062 -25736.303  24120.201 -23684.78 ]
(10,)
359089.228515625
Day 79 return is 1.652480207101483
gain/loss: 5933.879039353213, portfolio_value: 365023.1446083173


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/uni

[-50968.855  48469.473 -46319.72   40300.316 -39942.168  37200.004
  27066.055 -26161.592  24518.785 -24076.168]
(10,)
365023.134765625
Day 80 return is 1.6524803349653783
gain/loss: 6031.935682724678, portfolio_value: 371055.080291042
Final portfolio value: 371055.080291042


/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/univariate/base.py:766: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


corrected code
normal optimization - 592301

garch - 371055